# Advanced RAG on Hugging Face documentation using LangChain
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

This notebook demonstrates how you can build an advanced RAG (Retrieval Augmented Generation) for answering a user's question about a specific knowledge base (here, the HuggingFace documentation), using LangChain.

For an introduction to RAG, you can check [this other cookbook](rag_zephyr_langchain)!

RAG systems are complex, with many moving parts: here is a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

> 💡 As you can see, there are many steps to tune in this architecture: tuning the system properly will yield significant performance gains.

In this notebook, we will take a look into many of these blue notes to see how to tune your RAG system and get the best performance.

__Let's dig into the model building!__ First, we install the required model dependancies.

In [11]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [12]:
!node -v

/bin/bash: line 1: node: command not found


In [13]:
#!pip install --upgrade pip

In [14]:
#!pip install --upgrade Jupyter

In [15]:
!pip install --root-user-action ignore python-docx gradio pymilvus PyPDF2 langchain_huggingface plotly widgetsnbextension pandas-profiling torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [16]:
%reload_ext dotenv
%dotenv

cannot find .env file


In [17]:
from tqdm import tqdm
for i in tqdm(range(10000000)): pass

100%|██████████| 10000000/10000000 [00:01<00:00, 7567021.65it/s]


In [18]:
#from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option(
    "display.max_colwidth", None
)  # This will be helpful when visualizing retriever outputs

### Load your knowledge base

# Using langdocs to retrieve text 

# Using PDF files to retrieve text

# Using Text files

In [19]:
import os
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm

def load_text_to_langchain_docs(text_directory):
    RAW_KNOWLEDGE_BASE = []
    
    # Get all text files in the directory
    text_files = [f for f in os.listdir(text_directory) if f.endswith('.txt')]
    print('found the following text files:', text_files)
    
    for text_file in tqdm(text_files, desc="Processing text files"):
        file_path = os.path.join(text_directory, text_file)
        
        try:
            # Open and read the text file
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                
                if text.strip():  # Only add non-empty files
                    doc = LangchainDocument(
                        page_content=text,
                        metadata={
                            "source": file_path,
                            "file": text_file
                        }
                    )
                    RAW_KNOWLEDGE_BASE.append(doc)
        except Exception as e:
            print(f"Error processing {text_file}: {str(e)}")
    
    return RAW_KNOWLEDGE_BASE

# Usage
#text_directory = "./text_files"
#RAW_KNOWLEDGE_BASE = load_text_to_langchain_docs(text_directory)

#print(f"Total documents loaded: {len(RAW_KNOWLEDGE_BASE)}")

# Optional: Print the first few characters of the first document
#if RAW_KNOWLEDGE_BASE:
#    print("First document preview:")
#    print(RAW_KNOWLEDGE_BASE[0].page_content[:200])  # First 200 characters

In [20]:
RAW_KNOWLEDGE_BASE = []
#RAW_KNOWLEDGE_BASE = langdocs()
pdf_directory = "./pdf_files/"
RAW_KNOWLEDGE_BASE = RAW_KNOWLEDGE_BASE+load_pdf_to_langchain_docs(pdf_directory)
text_directory = "./text_files"
RAW_KNOWLEDGE_BASE = RAW_KNOWLEDGE_BASE+load_text_to_langchain_docs(text_directory)
word_directory = "./word_files"
RAW_KNOWLEDGE_BASE = RAW_KNOWLEDGE_BASE+load_word_to_langchain_docs(word_directory)
print(f"Total documents loaded: {len(RAW_KNOWLEDGE_BASE)}")
#RAW_KNOWLEDGE_BASE = RAW_KNOWLEDGE_BASE+langdocs()
# Optional: Print the first few characters of the first document
if RAW_KNOWLEDGE_BASE:
    print("First document preview:")
    print(RAW_KNOWLEDGE_BASE[-1].p`age_content[:200])  # First 200 characters

NameError: name 'load_pdf_to_langchain_docs' is not defined

# 1. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns a few relevant snippets from your knowledge base.

These snippets will then be fed to the Reader Model to help it generate its answer.

So __our objective here is, given a user question, to find the most relevant snippets from our knowledge base to answer that question.__

This is a wide objective, it leaves open some questions. How many snippets should we retrieve? This parameter will be named `top_k`.

How long should these snippets be? This is called the `chunk size`. There's no one-size-fits-all answers, but here are a few elements:
- 🔀 Your `chunk size` is allowed to vary from one snippet to the other.
- Since there will always be some noise in your retrieval, increasing the `top_k` increases the chance to get relevant elements in your retrieved snippets. 🎯 Shooting more arrows increases your probability of hitting your target.
- Meanwhile, the summed length of your retrieved documents should not be too high: for instance, for most current models 16k tokens will probably drown your Reader model in information due to [Lost-in-the-middle phenomenon](https://huggingface.co/papers/2307.03172). 🎯 Give your reader model only the most relevant insights, not a huge pile of books!


> In this notebook, we use Langchain library since __it offers a huge variety of options for vector databases and allows us to keep document metadata throughout the processing__.

### 1.1 Split the documents into chunks

- In this part, __we split the documents from our knowledge base into smaller chunks__ which will be the snippets on which the reader LLM will base its answer.
- The goal is to prepare a collection of **semantically relevant snippets**. So their size should be adapted to precise ideas: too small will truncate ideas, and too large will dilute them.

💡 _Many options exist for text splitting: splitting on words, on sentence boundaries, recursive chunking that processes documents in a tree-like way to preserve structure information... To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt._


- **Recursive chunking** breaks down the text into smaller parts step by step using a given list of separators sorted from the most important to the least important separator. If the first split doesn't give the right size or shape of chunks, the method repeats itself on the new chunks using a different separator. For instance with the list of separators `["\n\n", "\n", ".", ""]`:
    - The method will first break down the document wherever there is a double line break `"\n\n"`.
    - Resulting documents will be split again on simple line breaks `"\n"`, then on sentence ends `"."`.
    - Finally, if some chunks are still too big, they will be split whenever they overflow the maximum size.

- With this method, the global structure is well preserved, at the expense of getting slight variations in chunk size.

> [This space](https://huggingface.co/spaces/A-Roucher/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

🔬 Let's experiment a bit with chunk sizes, beginning with an arbitrary size, and see how splits work. We use Langchain's implementation of recursive chunking with `RecursiveCharacterTextSplitter`.
- Parameter `chunk_size` controls the length of individual chunks: this length is counted by default as the number of characters in the chunk.
- Parameter `chunk_overlap` lets adjacent chunks get a bit of overlap on each other. This reduces the probability that an idea could be cut in half by the split between two adjacent chunks. We ~arbitrarily set this to 1/10th of the chunk size, you could try different values!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNKSIZE = 256
OVERLAP = 10
# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNKSIZE,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=OVERLAP,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])


We also have to keep in mind that when embedding documents, we will use an embedding model that accepts a certain maximum sequence length `max_seq_length`.

So we should make sure that our chunk sizes are below this limit because any longer chunk will be truncated before processing, thus losing relevancy.

👀 As you can see, __the chunk lengths are not aligned with our limit of 512 tokens__, and some documents are above the limit, thus some part of them will be lost in truncation!
 - So we should change the `RecursiveCharacterTextSplitter` class to count length in number of tokens instead of number of characters.
 - Then we can choose a specific chunk size, here we would choose a lower threshold than 512:
    - smaller documents could allow the split to focus more on specific ideas.
    - But too small chunks would split sentences in half, thus losing meaning again: the proper tuning is a matter of balance.

➡️ Now the chunk length distribution looks better!

### 1.2 Building the vector database

We want to compute the embeddings for all the chunks of our knowledge base: to learn more about sentence embeddings, we recommend reading [this guide](https://osanseviero.github.io/hackerllama/blog/posts/sentence_embeddings/).

#### How does retrieval work?

Once the chunks are all embedded, we store them in a vector database. When the user types in a query, it gets embedded by the same model previously used, and a similarity search returns the closest documents from the vector database.

The technical challenge is thus, given a query vector, to quickly find the nearest neighbors of this vector in the vector database. To do this, we need to choose two things: a distance, and a search algorithm to find the nearest neighbors quickly within a database of thousands of records.

##### Nearest Neighbor search algorithm

There are plentiful choices for the nearest neighbor search algorithm: we go with Facebook's [FAISS](https://github.com/facebookresearch/faiss) since FAISS is performant enough for most use cases, and it is well known and thus widely implemented.

##### Distances

Regarding distances, you can find a good guide [here](https://osanseviero.github.io/hackerllama/blog/posts/sentence_embeddings/#distance-between-embeddings). In short:

- **Cosine similarity** computes the similarity between two vectors as the cosinus of their relative angle: it allows us to compare vector directions regardless of their magnitude. Using it requires normalizing all vectors, to rescale them into unit norm.
- **Dot product** takes into account magnitude, with the sometimes undesirable effect that increasing a vector's length will make it more similar to all others.
- **Euclidean distance** is the distance between the ends of vectors.

You can try [this small exercise](https://developers.google.com/machine-learning/clustering/similarity/check-your-understanding) to check your understanding of these concepts. But once vectors are normalized, [the choice of a specific distance does not matter much](https://platform.openai.com/docs/guides/embeddings/which-distance-function-should-i-use).

Our particular model works well with cosine similarity, so choose this distance, and we set it up both in the Embedding model, and in the `distance_strategy` argument of our FAISS index. With cosine similarity, we have to normalize our embeddings.

🚨👇 The cell below takes a few minutes to run on A10G!

👀 To visualize the search for the closest documents, let's project our embeddings from 384 dimensions down to 2 dimensions using PaCMAP.

💡 _We chose PaCMAP rather than other techniques such as t-SNE or UMAP, since [it is efficient (preserves local and global structure), robust to initialization parameters and fast](https://www.nature.com/articles/s42003-022-03628-x#Abs1)._

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/PaCMAP_embeddings.png" height="700">


➡️ On the graph above, you can see a spatial representation of the knowledge base documents. As the vector embeddings represent the document's meaning, their closeness in meaning should be reflected in their embedding's closeness.

The user query's embedding is also shown: we want to find the `k` documents that have the closest meaning, thus we pick the `k` closest vectors.

In the LangChain vector database implementation, this search operation is performed by the method `vector_database.similarity_search(query)`.

Here is the result:

# 2. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved context to formulate its answer.__

There are substeps that can all be tuned:
1. The content of the retrieved documents is aggregated together into the "context", with many processing options like _prompt compression_.
2. The context and the user query are aggregated into a prompt and then given to the LLM to generate its answer.

### 2.1. Reader model

The choice of a reader model is important in a few aspects:
- the reader model's `max_seq_length` must accommodate our prompt, which includes the context output by the retriever call: the context consists of 5 documents of 512 tokens each, so we aim for a context length of 4k tokens at least.
- the reader model

For this example, we chose [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a small but powerful model.

With many models being released every week, you may want to substitute this model to the latest and greatest. The best way to keep track of open source LLMs is to check the [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

To make inference faster, we will load the quantized version of the model:

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"
'''
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    
)
'''
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,  # Changed from bfloat16 to float16
    bnb_4bit_quant_storage=torch.uint8,    # Added for storage optimization
    use_nested_quant=True,                 # Added for nested quantization
)

model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
   
)

### 2.2. Prompt

The RAG prompt template below is what we will feed to the Reader LLM: it is important to have it formatted in the Reader LLM's chat template.

We give it our context and the user's question.

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Follow these steps to answer the question:
1. First, analyze the provided context carefully.
2. If the answer is found in the context:
   - Provide a comprehensive but concise answer
   - Include the source file name from the metadata
   - Start your response with "Based on the provided context:"

3. If the answer is NOT found in the context:
   - Clearly state "The provided context does not contain this information."
   - Then say "However, based on my knowledge:"
   - Provide an accurate answer from your own knowledge base
   - Be specific and detailed while maintaining conciseness

Always ensure your response is directly relevant to the question asked."""
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Question: {question}"""
    }
]

RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

Let's test our Reader on our previously retrieved documents!

In [ ]:
retrieved_docs_text = [
    doc.page_content for doc in retrieved_docs
]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join(
    [f"Document {doc.metadata['source']}:::\n" + doc.page_content for i, doc in enumerate(retrieved_docs)]
)

final_prompt = RAG_PROMPT_TEMPLATE.format(
    question="who is Samer?", context=context
)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
answer = READER_LLM(final_prompt)
print(answer)

### 2.3. Reranking

A good option for RAG is to retrieve more documents than you want in the end, then rerank the results with a more powerful retrieval model before keeping only the `top_k`.

For this, [Colbertv2](https://arxiv.org/abs/2112.01488) is a great choice: instead of a bi-encoder like our classical embedding models, it is a cross-encoder that computes more fine-grained interactions between the query tokens and each document's tokens.

It is easily usable thanks to [the RAGatouille library](https://github.com/bclavie/RAGatouille).

In [ ]:
from ragatouille import RAGPretrainedModel

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# 3. Assembling it all!

In [ ]:
from transformers import Pipeline


def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
#    reranker: Optional[RAGPretrainedModel] = None,
    reranker: RERANKER,
    num_retrieved_docs: int = 10,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )


    #relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, [doc.page_content for doc in relevant_docs], k=num_docs_final)
        print('reranked:',relevant_docs)
        #relevant_docs = [doc["content"] for doc in relevant_docs]
        relevant_docs = relevant_docs[:num_docs_final]
    

   

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(doc.metadata['source'])}:::\n" + doc.page_content for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

Let's see how our RAG pipeline answers a user query.

In [ ]:
import time

def measure_time(func):
    def microseconds_to_readable(microseconds):
        # Convert microseconds to seconds
        total_seconds = microseconds / 1_000_000

        # Calculate hours, minutes, and remaining seconds
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)

        # Format the result
        if hours > 0:
            return f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"
        elif minutes > 0:
            return f"{int(minutes)}m {seconds:.2f}s"
        else:
            return f"{seconds:.2f}s"
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        execution_time1 = (end_time - start_time) * 1_000_000  # Convert to microseconds
        execution_time = microseconds_to_readable(execution_time1)
        print(f"Function '{func.__name__}' took {execution_time}")
        return result
    return wrapper

In [ ]:
@measure_time
def ragthis(strvar, rankthis=1):
    if rankthis:
        reranker=RERANKER
    else:
        reranker=None
    answer3, relevant_docs3 = answer_with_rag(
        strvar, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=reranker
    )
    return answer3, relevant_docs3

In [ ]:
import sys
from collections import abc
import numpy as np

def get_size(obj, seen=None):
    """Recursively calculate size of objects in memory"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

def size_in_mb(obj):
    """Get size of object in MB"""
    bytes_size = get_size(obj)
    mb_size = bytes_size / (1024 * 1024)
    return f"{mb_size:.2f} MB"

# Example usage

print(f"Size of the RAW_KNOWLEDGE_BASE: {size_in_mb(RAW_KNOWLEDGE_BASE)}")


print(f"Size of the docs_processed: {size_in_mb(docs_processed)}")
print(f"Size of KNOWLEDGE_VECTOR_DATABASE: {size_in_mb(KNOWLEDGE_VECTOR_DATABASE)}")

In [ ]:
import re
from langchain.docstore.document import Document as LangchainDocument

def count_words(text):
    # Split the text into words using regex
    # This considers any sequence of non-whitespace characters as a word
    words = re.findall(r'\S+', text)
    return len(words)

def analyze_knowledge_base(documents):
    total_chars = 0
    total_words = 0
    
    for doc in documents:
        if isinstance(doc, LangchainDocument):
            content = doc.page_content
            total_chars += len(content)
            total_words += count_words(content)
    
    return total_chars, total_words

# Assuming RAW_KNOWLEDGE_BASE is already defined
total_characters, total_words = analyze_knowledge_base(RAW_KNOWLEDGE_BASE)

print(f"Total number of characters: {'{:,d}'.format(total_characters)}")
print(f"Total number of words: {'{:,d}'.format(total_words)}")

# If you want to know the average per document:
num_documents = len(RAW_KNOWLEDGE_BASE)
avg_characters = total_characters / num_documents
avg_words = total_words / num_documents

print(f"Number of documents: {'{:,d}'.format(num_documents)}")
#print(f"Average characters per document: {avg_characters:.2f}")
#print(f"Average words per document: {avg_words:.2f}")

In [ ]:

print(f"Total number of characters: {'{:,d}'.format(total_characters)}")
print(f"Total number of words: {'{:,d}'.format(total_words)}")

# If you want to know the average per document:
num_documents = len(docs_processed)

print(f"Number of documents: {'{:,d}'.format(num_documents)}")
print(f"Size of the RAW_KNOWLEDGE_BASE: {size_in_mb(RAW_KNOWLEDGE_BASE)}")
print(f"Size of the docs_processed: {size_in_mb(docs_processed)}")
print(f"Size of KNOWLEDGE_VECTOR_DATABASE: {size_in_mb(KNOWLEDGE_VECTOR_DATABASE)}")

In [ ]:
for doc in docs_processed:
    #print(doc.metadata.keys())
    if 'Windows' in str(doc.metadata['source']):
        #print(doc.page_content)
        continue
        #print('here')

In [ ]:
import gradio as gr
import random
try:
    demo.close()
except:
    pass
    # Optional: Add custom CSS for additional styling
custom_css = """
.container {
    max-width: 1000px;
    margin: auto;
    padding: 20px;
}
.footer {
    text-align: center;
    margin-top: 30px;
    padding: 20px;
}
"""
def close_server():
    try:
        print("Shutting down server...")
        demo.close()
    except Exception as e:
        print(f"Error while shutting down: {e}")
        #import os
        #os._exit(0)
def process_rag_query(question2):
    # Call your existing RAG function
    answer2, relevant_docs2 = ragthis(question2,rankthis=0)
    
    # Format the relevant documents for display
    docs_formatted = "\n\n".join([
        f"Document {i+1}:\n{doc}" for i, doc in enumerate(relevant_docs2)
    ])
    
    return answer2, docs_formatted

# Example questions for the interface
example_questions = [
    ["Who is Samer ?"],
    ["Who is Moataz ?"],
    ["what is the 'ready for test' status ?"],
]

with gr.Blocks(theme=gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="slate",
), css=custom_css) as demo:
    
    # Header section
    with gr.Row(elem_classes="container"):
        gr.Markdown("""
            # 🤖 AI Question Answering System By Moataz Allam
            #### Get instant answers powered Advanced RAG using three models for:
            ######    inferencing, ranking and embeddings. FAISS DB.
            ---
            """)
    
    # Main content
    with gr.Row(elem_classes="container"):
        with gr.Column(scale=2):
            # Input section
            with gr.Group():
                gr.Markdown("### Ask your question")
                question_input = gr.Textbox(
                    placeholder="Type your question here...",
                    label="Question",
                    lines=3,
                    scale=3
                )
                
                submit_btn = gr.Button(
                    "Get Answer",
                    variant="primary",
                    size="lg"
                )
                
                gr.Examples(
                    examples=example_questions,
                    inputs=question_input,
                    label="Try these example questions"
                )
    
    # Output section
    with gr.Row(elem_classes="container"):
        with gr.Column():
            gr.Markdown("### Results")
            with gr.Row():
                with gr.Column(scale=1):
                    answer_output = gr.Textbox(
                        label="📝 Answer",
                        lines=5,
                        show_copy_button=True
                    )
                
            with gr.Accordion("View Source Documents", open=False):
                docs_output = gr.Textbox(
                    label="📚 Reference Documents",
                    lines=8,
                    show_copy_button=True
                )
    
    # Footer
    with gr.Row(elem_classes="footer"):
        gr.Markdown("""
            ---
            *Powered by RAG Technology*
            """)
    
    # Handle the submit button click
    submit_btn.click(
        fn=process_rag_query,
        inputs=[question_input],
        outputs=[answer_output, docs_output],
        api_name="answer"
    )
    stop_btn = gr.Button("Stop Server")
    stop_btn.click(fn=close_server, inputs=None, outputs=None)

# Launch the interface
#if __name__ == "__main__":
#    demo.launch(
#        share=True,
#        server_name="0.0.0.0",
#        server_port=7860
#    )

In [ ]:
demo.close()
demo.launch(share=True, server_port=7862)

In [ ]:

#question = "Answer in not more than 10 words, what is the R660?"
question = "Answer by 'Yes' or 'No'. Is the R660 a pc?"
#question = "how much is maximum memory in the R660 ?, and who is Samer ?"
#question = 'what settings should I put in the virtualbox for the windows VM ?'
#question = 'when to move the status to ready for test ?'
answer, relevant_docs = ragthis(question,rankthis=0)
torch.cuda.empty_cache()

## 

In [ ]:
print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
fileset = set()
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc.metadata['source'])

### ✅ We now have a fully functional, performant RAG system. That's it for today! Congratulations for making it to the end 🥳


# To go further 🗺️

This is not the end of the journey! You can try many steps to improve your RAG system. We recommend doing so in an iterative way: bring small changes to the system and see what improves performance.

### Setting up an evaluation pipeline

- 💬 "You cannot improve the model performance that you do not measure", said Gandhi... or at least Llama2 told me he said it. Anyway, you should absolutely start by measuring performance: this means building a small evaluation dataset, and then monitor the performance of your RAG system on this evaluation dataset.

### Improving the retriever

🛠️ __You can use these options to tune the results:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

👷‍♀️ __More could be considered:__
- Try another chunking method, like semantic chunking
- Change the index used (here, FAISS)
- Query expansion: reformulate the user query in slightly different ways to retrieve more documents.

### Improving the reader

🛠️ __Here you can try the following options to improve results:__
- Tune the prompt
- Switch reranking on/off
- Choose a more powerful reader model

💡 __Many options could be considered here to further improve the results:__
- Compress the retrieved context to keep only the most relevant parts to answer the query.
- Extend the RAG system to make it more user-friendly:
    - cite source
    - make conversational